# Model Downloader (URLs only)

Single + batch URL downloader for ComfyUI model folders.

Optional environment variables:
- CIVITAI_TOKEN
- HF_TOKEN


In [ ]:
from pathlib import Path

# ComfyUI root
COMFY = Path('/workspace/ComfyUI')
MODELS = COMFY / 'models'

folders = {
    'checkpoints': MODELS / 'checkpoints',
    'loras': MODELS / 'loras',
    'vae': MODELS / 'vae',
    'controlnet': MODELS / 'controlnet',
    'ipadapter': MODELS / 'ipadapter',
    'clip_vision': MODELS / 'clip_vision',
    'text_encoders': MODELS / 'text_encoders',
    'diffusion_models': MODELS / 'diffusion_models',
    'unet': MODELS / 'unet'
}

for p in folders.values():
    p.mkdir(parents=True, exist_ok=True)

folders


In [ ]:
%pip -q install "requests>=2.31" "tqdm>=4.66" "ipywidgets>=8.1"


In [ ]:
import os
print('CIVITAI_TOKEN set?', bool(os.environ.get('CIVITAI_TOKEN')))
print('HF_TOKEN set?', bool(os.environ.get('HF_TOKEN')))


In [ ]:
import os
import re
from pathlib import Path
from urllib.parse import unquote

def _filename_from_cd(cd):
    """Extract filename from Content-Disposition header."""
    if not cd:
        return None

    # RFC 5987: filename*=UTF-8''encoded
    m = re.search(r"filename\*\s*=\s*UTF-8''([^;]+)", cd, flags=re.I)
    if m:
        return Path(unquote(m.group(1).strip().strip('"'))).name

    # filename="name.ext"
    m = re.search(r'filename\s*=\s*"([^"]+)"', cd, flags=re.I)
    if m:
        return Path(m.group(1)).name

    # filename=name.ext
    m = re.search(r"filename\s*=\s*([^;]+)", cd, flags=re.I)
    if m:
        return Path(m.group(1).strip().strip('"')).name

    return None

def download(url, folder_key='checkpoints', filename=None, overwrite=False):
    """Download a model file to ComfyUI/models/<folder_key>.
    Self-contained imports so widget callbacks don't break if cells run out of order.
    Supports resume via .part files.
    """
    import requests
    from tqdm.auto import tqdm

    if folder_key not in folders:
        raise ValueError(f"folder_key must be one of: {list(folders.keys())}")

    dest_dir = folders[folder_key]
    dest_dir.mkdir(parents=True, exist_ok=True)

    headers = {}
    civ = os.environ.get('CIVITAI_TOKEN')
    if 'civitai.com' in url and civ:
        headers['Authorization'] = f'Bearer {civ}'

    hf = os.environ.get('HF_TOKEN')
    if ('huggingface.co' in url or 'hf.co' in url) and hf:
        headers.setdefault('Authorization', f'Bearer {hf}')

    # Probe for filename/size
    with requests.get(url, headers=headers, stream=True, allow_redirects=True) as r:
        r.raise_for_status()

        if not filename:
            filename = _filename_from_cd(r.headers.get('content-disposition'))
        if not filename:
            filename = url.split('?')[0].rstrip('/').split('/')[-1] or 'download.bin'

        filename = filename.replace('\\\\', '_').replace('/', '_')
        dest = dest_dir / filename
        tmp = dest.with_suffix(dest.suffix + '.part')

        if dest.exists() and not overwrite:
            print(f'Already exists: {dest}')
            return dest
        if dest.exists() and overwrite:
            dest.unlink(missing_ok=True)

        existing = tmp.stat().st_size if tmp.exists() else 0
        cl = r.headers.get('content-length')
        total = (int(cl) + existing) if (cl and existing) else (int(cl) if cl else None)

    # Resume request
    if existing > 0:
        headers2 = dict(headers)
        headers2['Range'] = f'bytes={existing}-'
        rr = requests.get(url, headers=headers2, stream=True, allow_redirects=True)
        if rr.status_code not in (206, 200):
            rr.close()
            existing = 0
            rr = requests.get(url, headers=headers, stream=True, allow_redirects=True)
    else:
        rr = requests.get(url, headers=headers, stream=True, allow_redirects=True)

    with rr as r2:
        r2.raise_for_status()
        mode = 'ab' if existing > 0 else 'wb'
        with open(tmp, mode) as f, tqdm(total=total, initial=existing, unit='B', unit_scale=True, desc=filename) as pbar:
            for chunk in r2.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

    tmp.rename(dest)
    print(f'Saved: {dest}')
    return dest


## Single URL Downloader
Paste a URL, choose a destination folder, click Download.


In [ ]:
import ipywidgets as w
from IPython.display import display, clear_output

folder_dd = w.Dropdown(options=list(folders.keys()), value='checkpoints', description='Folder:')
url_tb = w.Text(value='', placeholder='Paste direct download URL…', description='URL:', layout=w.Layout(width='900px'))
name_tb = w.Text(value='', placeholder='Optional filename override', description='Name:', layout=w.Layout(width='900px'))
overwrite_cb = w.Checkbox(value=False, description='Overwrite')
btn = w.Button(description='Download', button_style='success')
out = w.Output()

def _go(_):
    url = url_tb.value.strip()
    if not url:
        with out:
            clear_output(wait=True)
            print('Paste a URL first.')
        return
    name = name_tb.value.strip() or None
    with out:
        clear_output(wait=True)
        print('Downloading to:', folders[folder_dd.value])
        print('URL:', url)
        download(url, folder_key=folder_dd.value, filename=name, overwrite=overwrite_cb.value)

btn.on_click(_go)
display(w.VBox([w.HBox([folder_dd, overwrite_cb]), url_tb, name_tb, btn, out]))


## Batch URL Downloader
One URL per line. Lines starting with `#` are ignored.


In [ ]:
batch_tb = w.Textarea(value='', placeholder='One URL per line', description='URLs:', layout=w.Layout(width='900px', height='180px'))
btn_batch = w.Button(description='Download all', button_style='success')
out2 = w.Output()

def _parse(text):
    urls = []
    for line in text.splitlines():
        s = line.strip()
        if not s or s.startswith('#'):
            continue
        urls.append(s)
    return urls

def _batch(_):
    urls = _parse(batch_tb.value)
    with out2:
        clear_output(wait=True)
        if not urls:
            print('Paste at least one URL.')
            return
        print(f'Downloading {len(urls)} file(s) to:', folders[folder_dd.value])
        fails = 0
        for i, u in enumerate(urls, start=1):
            try:
                print(f'[{i}/{len(urls)}] {u}')
                download(u, folder_key=folder_dd.value, overwrite=overwrite_cb.value)
            except Exception as e:
                fails += 1
                print('FAILED:', e)
        print('Done. Failures:', fails)

btn_batch.on_click(_batch)
display(w.VBox([batch_tb, btn_batch, out2]))
